# MCMC CALIBRATION TECHNICS IN CONTEXT OF  INFECTIOUS DISEASE MODELING

## Prerequies

In [1]:
# pip install multiprocess
# %pip install --upgrade --force-reinstall multiprocess

#Compatible with latest jax version  
# %pip install summerepi2==1.3.6
# %pip install jinja2

In [1]:
import multiprocess as mp
import platform

# This is required for pymc parallel evaluation in notebooks 
# But has to be change while using a python script 
# Use this following instruction instead
# if __name__ == "__main__":
#     if platform.system() != "Windows":
#         mp.set_start_method('spawn')
    
    # rest of your code body here inside the if __name__
if platform.system() != "Windows":
    
    mp.set_start_method('forkserver')

In [2]:
import Calibrate as cal #Runing the calibration process and gathering results
from calibs_utilities import get_all_priors, get_targets, load_data
from models.models import model1, model2, bcm_seir_age_strat, bcm_sir #All the models we design for the test
from Calibrate import plot_comparison_bars

# Combining tagets and prior with our summer2 model in a BayesianCompartmentalModel (bcm_model_1)
from estival.model import BayesianCompartmentalModel
from estival.sampling.tools import likelihood_extras_for_idata
from estival.sampling.tools import likelihood_extras_for_samples


import pandas as pd
import numpy as np
# import plotly.express as px
import matplotlib.pyplot as plt
from typing import List

import pymc as pm

# We use estivals parallel tools to run the model evaluations
from estival.utils.parallel import map_parallel

import numpyro
from numpyro import distributions as dist
from numpyro import infer
import arviz as az
import pickle
from datetime import datetime
from plotly import graph_objects as go
# import jax
from jax import numpy as jnp
pd.options.plotting.backend = "plotly" #To allow plotly graphic. Swich to "matplotlib" if facing some troubles while ploting
# pd.options.plotting.backend = "matplotlib"

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


# Application 1: The basic SIR model

## Model Definition and Configuration

A mechanistic model (ODE-Based) model discribing Infectious Disease transmission.

In [ ]:
model_1 = model1() 

In [ ]:
#Defining  a Bayesian Compartmental Model
#Targets and priors are already defined with the data 
#See models.py for the costumization
bcm_model_1 = bcm_sir()
# bcm_model_1 = BayesianCompartmentalModel(model_1, parameters, priors, targets)


### Trial run 

In [ ]:
pd.options.plotting.backend = "plotly" #To allow plotly graphic. Swich to "matplotlib" if facing some troubles while ploting
output_labels = {"index": "time", "value": "number infectious"}

model_1.run(bcm_model_1.parameters)
plt.rcParams["figure.figsize"] = (5, 5)


df = pd.DataFrame(
    {
        "modelled": model_1.get_outputs_df()["I"],
        "observed": bcm_model_1.targets["active_cases"].data,
    }
)
df.plot(kind="scatter", labels=output_labels) #,figsize=(3,3));

### Sampling 

In [ ]:
##____Uniform Initialisation_________
def init_uniform(num_chains, parameters):
    init_vals = []
    for c in range(num_chains):
        init_vals.append({param: np.random.uniform(0.0,1.0) for param in parameters.keys()})
    
    return init_vals


init_vals_nuts = {param: jnp.array(np.random.uniform(0.0,1.0, 4)) for param in bcm_model_1.parameters.keys()}

init_vals_4 = init_uniform(4,bcm_model_1.parameters)
init_vals_6 = init_uniform(6,bcm_model_1.parameters)



In [ ]:
#Defining the Numpyro model using our likelihood from the BaysianCompartmentalModel
def nmodel():
    sampled = {k:numpyro.sample(k, dist.Uniform(0.0,1.0)) for k in bcm_model_1.parameters}
    ll = numpyro.factor("ll", bcm_model_1.loglikelihood(**sampled))

#### Simple Run

In [ ]:
%%time
D = 2 # Dimension of the parameter's space
samplers =  [infer.NUTS] + [pm.sample_smc] + [pm.Metropolis] + [pm.DEMetropolisZ] + [pm.DEMetropolis]*2
Draws = [2000] + [2000] + [10000] + [8000]*3
# Tunes = [0] + [100, 1000]*5
Init = [init_vals_nuts] + [init_vals_4]*4 + [init_vals_6]
Chains = [4]*5 + [6]
results_df = pd.DataFrame()

for sampler, draws, chains, init in zip(samplers, Draws, Chains, Init):
    # if sampler.__name__ == "NUTS":
    #     init = init_vals_nuts
    # else:
    #     init = init_vals_4
    results = cal.Single_analysis(sampler = sampler, 
            draws = draws,
            chains=chains,
            cores = chains,
            tune = 1000,
            bcm_model = bcm_model_1,
            # n_iterations = 1,
            nmodel=nmodel,
            # n_jobs = 1,
            initial_params = init

    )
            
    results_df = pd.concat([results_df,results])



results_df = results_df.reset_index(drop=True)


In [ ]:
results_df
results_df.style.set_caption("MCMC COMPARISON") 


In [ ]:

#Storing results on a pickle file
with open('./Results/Model_1/Simple_run_results_3.pkl', 'wb') as fp:
    pickle.dump(results_df, fp)

#### Multiple runs

In [ ]:
all_results = dict()

In [ ]:
sampler = pm.sample_smc
all_results[sampler.__name__] = cal.multirun(
    sampler, 
    draws = 10000,
    tune = 1000,
    chains=4,
    cores=4, 
    bcm_model = bcm_model_1,
    n_iterations = 100,
    n_jobs = 3,    
    )

In [ ]:
sampler = pm.DEMetropolisZ
all_results[sampler.__name__] = cal.multirun(
    sampler, 
    draws = 10000,
    chains=4,
    cores=4,
    tune = 1000, 
    bcm_model = bcm_model_1,
    n_iterations = 100,
    n_jobs = 3,
    )

In [ ]:
sampler = pm.Metropolis
all_results[sampler.__name__] = cal.multirun(
    sampler, 
    draws = 10000,
    tune = 1000, 
    chains=4,
    cores=4,
    bcm_model = bcm_model_1,
    n_iterations = 100,
    n_jobs = 3,
    )

In [ ]:
sampler = infer.NUTS
all_results[sampler.__name__] = cal.multirun(
    sampler, 
    draws = 10000,
    tune = 1000,
    chains=4,
    cores=4, 
    bcm_model = bcm_model_1,
    nmodel=nmodel,
    n_iterations = 100,
    n_jobs = 2,
    )

In [ ]:
sampler = pm.DEMetropolis
all_results[sampler.__name__] = cal.multirun(
    sampler, 
    draws = 1000,
    tune = 1000,
    chains=6,
    cores=4, 
    bcm_model = bcm_model_1,
    n_iterations = 100,
    n_jobs = 3,
    )

In [ ]:
all_results["DEMetropolis"]

In [ ]:
#Storing the results for later analysis

with open('./Results/Model_1/Multi_run_results_without_init_1.pkl', 'wb') as fp:
    pickle.dump(all_results, fp)


#### Summarizing the 100 results

We call the function group_summary from the calibrate modules. This will help to figure out the average performance
of sampler over 100 runs

In [ ]:
#Loading a pickle file
with open('./Results/Model_1/Multi_run_results_3.pkl', 'rb') as fp:
    multi_res = pickle.load(fp) #It's a dict
df = pd.concat(multi_res) #converting to dataframe

Here best results refers to the run with the min 'Max_rhat' for each sampler over the  runs

In [ ]:
best_results = pd.DataFrame()
for sampler in multi_res.keys():
    temp = multi_res[sampler].round(3)
    temp
    best_rhat = temp.loc[[temp["Rhat_max"].idxmin()]]
    best_results = pd.concat([best_results,best_rhat])

In [ ]:
best_results = best_results.reset_index(drop=True)
#Computing the Relative ESS
best_results["Rel_Ess"] = best_results['Min_Ess'].astype(float)/(best_results["Draws"].astype(float)*best_results['Chains'].astype(float))


In [ ]:
summaries_mean, prcnt_succ = cal.group_summary(df)

In [ ]:
summaries_mean.round(2).to_latex(column_format="|c|c|c|c|c|c|c|c|c|c|c|")

In [ ]:
prcnt_succ = prcnt_succ.round(2)

In [ ]:
prcnt_succ#.to_latex(column_format='|c|c|')

#### Using arviz for trace visualization

In [ ]:
for idata, sampler in zip(best_results.Trace, best_results.Sampler):
    print(sampler)
    az.plot_rank(idata,figsize=(9,4))
    plt.show()

#### Bar Ploting Comparison

In [ ]:
# plot_comparison_Bars(results_df=res)
best_results["Run"] = best_results.Sampler + "\nDraws=" + best_results.Draws.astype(str) + "\nTune=" + best_results.Tune.astype(str) +"\nChains=" + best_results.Chains.astype(str)
plot_comparison_bars(best_results.round(2))

### Fitting test

Here we test if the model is well fitted to the data, we will use the results from the single run

In [ ]:
IDATA = best_results["Trace"]

In [ ]:
map_res = dict()
for idata, sampler in zip(IDATA,best_results["Sampler"]):
    map_res[sampler] = cal.fitting_test(idata, bcm_model_1, model_1)

In [ ]:
pd.options.plotting.backend = "plotly" #To allow plotly graphic. Swich to "matplotlib" if facing some troubles while ploting

NUTS = map_res["NUTS"]["I"]
SMC = map_res["sample_smc"]["I"]
DEM = map_res["DEMetropolis"]["I"]
MH = map_res["Metropolis"]["I"]
DEMZ = map_res["DEMetropolisZ"]["I"]



df = pd.DataFrame(
    {   
        "MH": MH,
        "NUTS": NUTS,
        "SMC": SMC,
        "DEM": DEM,
        "DEMZ": DEMZ,
        "observed": bcm_model_1.targets["active_cases"].data,
    }
)
df.plot(kind="scatter", labels=output_labels, title="Model fitting")#

## Uncertainty sampling

In [ ]:
# Wrapper function captures our bcm from the main namespace to pass into map_parallel
# Using this idiom in closures/factory functions is typical
def run_sample(idx_sample):
    idx, params = idx_sample
    return idx, bcm_model_1.run(params)

# Run the samples through our BCM using the above function
# map_parallel takes a function and an iterable as input

# We use 4 workers here, default is cpu_count/2 (assumes hyperthreading)
sample_res = dict()
for idata,sampler in zip(IDATA,best_results["Sampler"]):
    sample_idata = az.extract(idata, num_samples=4000)
    samples_df = sample_idata.to_dataframe().drop(columns=["chain","draw"])
    sample_res[sampler] = map_parallel(run_sample, samples_df.iterrows(), n_workers=4)

In [ ]:
# We'll use xarray for this step; aside from computing things very quickly, it's useful
# to persist the run results to netcdf/zarr etc

import xarray as xr

In [ ]:
# Build a DataArray out of our results, then assign coords for indexing
xres = xr.DataArray(np.stack([r.derived_outputs for idx, r in sample_res["NUTS"]]), 
                    dims=["sample","time","variable"])
xres = xres.assign_coords(sample=sample_idata.coords["sample"], 
                          time=map_res["NUTS"].index, variable=pd.DataFrame(map_res["NUTS"]["I"]).columns)

In [ ]:
# Set some quantiles to calculate
quantiles = (0.5,0.75,0.95)

# Generate a new DataArray containing the quantiles
xquantiles = xres.quantile(quantiles,dim=["sample"])

In [ ]:
# Extract these values to a pandas DataFrame for ease of plotting

uncertainty_df = xquantiles.to_dataframe(name="value").reset_index().set_index("time").pivot(columns=("variable","quantile"))["value"]

In [ ]:
variable = "active_cases"
pd.options.plotting.backend = "matplotlib" #To allow plotly graphic. Swich to "matplotlib" if facing some troubles while ploting

fig = uncertainty_df["I"].plot.area(title=variable,alpha=0.7)
pd.Series(map_res["NUTS"]["I"]).plot(label = "modelled",style='--')
bcm_model_1.targets[variable].data.plot(label = "observed",style='.',color="black", ms=5, alpha=0.8)
plt.legend()

# Application 2: The SEIR age-stratified model

## Data for fitting
Here we will define a target for each age category

In [3]:
pd.options.plotting.backend = "plotly" #To allow plotly graphic. Swich to "matplotlib" if facing some troubles while ploting

df = pd.DataFrame()
df = pd.read_csv("./data/new_cases_England_2020.csv")
df["date"] = pd.to_datetime(df.date)
df.set_index(["age","date"], inplace=True)



In [7]:
total_cases = df.groupby("date").sum()
#Rolling by 14 days to discard fluctuations
total_cases = total_cases.rolling(14).mean().iloc[14:]
# D = total_cases["Jun 2020":"Nov 2020"].iloc[::5]

## Model Definition


In [4]:
model_config = {"compartments": ("S", "E","I","R"), # "Ip","Ic", "Is", "R"),
        "population": 56490045, #England population size 2021
        "start_time": datetime(2020, 6, 1),
        "end_time": datetime(2020, 11, 30)
}
bcm_model_2 = bcm_seir_age_strat(model_config)
model_2 = model2(model_config)

In [17]:
bcm_model_2.parameters

{'age_transmission_rate_0': 0.25,
 'age_transmission_rate_5': 0.25,
 'age_transmission_rate_10': 0.25,
 'age_transmission_rate_15': 0.25,
 'age_transmission_rate_20': 0.25,
 'age_transmission_rate_25': 0.25,
 'age_transmission_rate_30': 0.25,
 'age_transmission_rate_35': 0.25,
 'age_transmission_rate_40': 0.25,
 'age_transmission_rate_45': 0.25,
 'age_transmission_rate_50': 0.25,
 'age_transmission_rate_55': 0.25,
 'age_transmission_rate_60': 0.25,
 'seed': 100.0,
 'incubation_period': 5.4,
 'infectious_period': 7.3}

## Trial run

In [5]:
disp_params = {k:v.ppf(0.5) for k,v in bcm_model_2.priors.items() if "_disp" in k} #Mandatory if you tempt to calibrate your target dispersion

res = bcm_model_2.run(bcm_model_2.parameters | disp_params).derived_outputs

# res = model_2.get_outputs_df()
Infec = [f"IXage_{i}" for i in range(0,65,5)] #Selecting the Infectious compartments
#Summing over the infectious compartments 
total_cases_pred = pd.DataFrame(res[Infec].sum(axis=1))

In [8]:

plot_start_date = datetime(2020, 8, 1)
analysis_end_date = datetime(2020, 11, 30)

# plot = model_2.get_outputs_df()["IXage_60"].plot()
plot = pd.DataFrame(total_cases_pred).plot()
plot.update_xaxes(range=(plot_start_date, analysis_end_date))
plot.add_trace(go.Scatter(x=total_cases.index, y=total_cases["cases"], mode='markers', name='total_cases'))
#pd.options.plotting.backend = "plotly" #To allow plotly graphic. Swich to "matplotlib" if facing some troubles while ploting
#pivot_df["total_cases"].plot.area()

## Testing by optimization

In [9]:
import nevergrad as ng
import pandas as pd 
import numpy as np
# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model

In [10]:
# TwoPointsDE is a good suggested default for some of our typical use cases
opt_class = ng.optimizers.TwoPointsDE
orunner = optimize_model(bcm_model_2, opt_class=opt_class)
# Here we run the optimizer in a loop, inspecting the current best point at each iteration
# Using the loss information at each step can provide the basis for stopping conditions

# for i in range(8):
    # Run the minimizer for a specified 'budget' (ie number of evaluations)
rec = orunner.minimize(1000)
    # Print the loss (objective function value) of the current recommended parameters
# print(rec.loss)
mle_params = rec.value[1]
res_opt = bcm_model_2.run(mle_params)

In [13]:
total_cases_pred_opt = pd.DataFrame(res_opt.derived_outputs[Infec].sum(axis=1))

In [14]:
D = total_cases["Aug 2020":"Nov 2020"]

In [16]:
plot_start_date = datetime(2020, 8, 1)
analysis_end_date = datetime(2020, 11, 30)

# plot = model_2.get_outputs_df()["IXage_60"].plot()
plot = pd.DataFrame(total_cases_pred_opt).plot()
plot.update_xaxes(range=(plot_start_date, analysis_end_date))
plot.add_trace(go.Scatter(x=D.index, y=D["cases"], mode='markers', name='total_cases'))

## Calibration

In [ ]:
##____Uniform Initialisation for each chain_________
chains = 4
init_vals = []
for c in range(chains):
    temp = {param: np.random.uniform(0.0,1.0) for param in list(bcm_model_2.parameters.keys())[:-2]}
    temp["incubation_period"] = np.random.uniform(1.,15.) 
    temp["infectious_period"] = np.random.uniform(1.,15.)
    init_vals.append(temp)


init_vals_nuts = {param: jnp.array(np.random.uniform(0.0,1.0, 4)) for param in list(bcm_model_2.parameters)[:-2]}
init_vals_nuts["incubation_period"] = jnp.array(np.random.uniform(1.,15.0, 4))
init_vals_nuts["infectious_period"] = jnp.array(np.random.uniform(1.,15.0, 4))

In [18]:
def nmodel_2():
    unif_priors = list(bcm_model_2.parameters)[:-2]
    sampled = {k:numpyro.sample(k, dist.Uniform(0.0,1.0)) for k in unif_priors}
    sampled["seed"] = numpyro.sample("seed", dist.Uniform(1.0,1000))
    #Adding the normal priors for the incubation and infectious periods

    # sampled["incubation_period"] = numpyro.sample("incubation_period", dist.TruncatedNormal(7.3, 2.0, low=1., high=14.))
    # sampled["infectious_period"] = numpyro.sample("infectious_period", dist.TruncatedNormal(5.4, 3.0, low=1., high=14.))
    # Log-likelihood
    disp_params = {k:v.ppf(0.5) for k,v in bcm_model_2.priors.items() if "_disp" in k}

    log_likelihood = bcm_model_2.loglikelihood(**sampled | disp_params)

    numpyro.factor("ll",log_likelihood)

### Performing a Single run for each algorithm

In [25]:

samplers =  [pm.sample_smc,pm.Metropolis,pm.DEMetropolisZ,pm.DEMetropolis]
Draws = [5000]*4
# Init = [init_vals_4]*4
results_df = pd.DataFrame()

for sampler, draws in zip(samplers, Draws):#, Init):
    
    results = cal.Single_analysis(sampler = sampler, 
            draws = draws,
            chains=4,
            cores = 4,
            tune = 1000,
            bcm_model = bcm_model_2,
            # initial_params = init
            )
            
    results_df = pd.concat([results_df,results])



results_df = results_df.reset_index(drop=True)

Initializing SMC sampler...
Sampling 4 chains in 4 jobs


The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [seed]
>Metropolis: [age_transmission_rate_0]
>Metropolis: [age_transmission_rate_5]
>Metropolis: [age_transmission_rate_10]
>Metropolis: [age_transmission_rate_15]
>Metropolis: [age_transmission_rate_20]
>Metropolis: [age_transmission_rate_25]
>Metropolis: [age_transmission_rate_30]
>Metropolis: [age_transmission_rate_35]
>Metropolis: [age_transmission_rate_40]
>Metropolis: [age_transmission_rate_45]
>Metropolis: [age_transmission_rate_50]
>Metropolis: [age_transmission_rate_55]
>Metropolis: [age_transmission_rate_60]
Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_00

In [27]:
Trace = results_df["Trace"]

In [30]:
idata = Trace[0]

In [33]:
az.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
seed,341.365,312.356,2.868,915.418,74.095,53.278,16.0,16.0,1.17
age_transmission_rate_0,0.375,0.301,0.013,0.937,0.069,0.052,24.0,76.0,1.24
age_transmission_rate_5,0.322,0.302,0.001,0.936,0.059,0.052,174.0,16.0,1.27
age_transmission_rate_10,0.389,0.315,0.002,0.915,0.083,0.060,14.0,30.0,1.20
age_transmission_rate_15,0.350,0.289,0.001,0.890,0.060,0.043,29.0,78.0,1.09
age_transmission_rate_20,0.506,0.296,0.100,0.999,0.111,0.082,7.0,79.0,1.49
age_transmission_rate_25,0.316,0.314,0.003,0.927,0.065,0.054,73.0,66.0,1.21
age_transmission_rate_30,0.377,0.277,0.017,0.915,0.080,0.058,12.0,89.0,1.24
age_transmission_rate_35,0.411,0.280,0.017,0.943,0.052,0.043,30.0,52.0,1.21
age_transmission_rate_40,0.356,0.304,0.008,0.924,0.086,0.062,12.0,69.0,1.23


In [ ]:
#Probabily the nuts will take a while to finish--We need to reduce the number of iterations and or warmup
res_nuts = cal.Single_analysis(sampler = infer.NUTS, 
            draws = 1000,
            tune = 10,
            chains = 4,
            cores=4,
            bcm_model = bcm_model_2,
            nmodel=nmodel_2,
            initial_params = init_vals_nuts)

results_df = pd.concat([results_df,res_nuts])


In [36]:
def fitting_test(sampler,idata, bcm, model):
    from estival.sampling.tools import likelihood_extras_for_samples,likelihood_extras_for_idata
    if sampler.__name__ in ["DEMetropolis", "DEMetropolisZ"]:
        likelihood_df = likelihood_extras_for_idata(idata, bcm) #More faster
    else :
        likelihood_df = likelihood_extras_for_samples(idata.posterior, bcm)

    ldf_sorted = likelihood_df.sort_values(by="logposterior",ascending=False)

    # Extract the parameters from the calibration samples
    map_params = idata.posterior.to_dataframe().loc[ldf_sorted.index[0]].to_dict()
    map_params['incubation_period']= 5.4
    map_params['infectious_period'] = 7.3
    print(map_params)
    bcm.loglikelihood(**map_params), ldf_sorted.iloc[0]["loglikelihood"]
    # Run the model with these parameters
    model.run(map_params)
    # ...and plot some results
    return model.get_outputs_df()

In [37]:
map_res_smc = fitting_test(pm.sample_smc,idata, bcm_model_2,model_2)

{'seed': 204.54780352014805, 'age_transmission_rate_0': 0.008861581952759478, 'age_transmission_rate_5': 0.0035240977332825197, 'age_transmission_rate_10': 0.041556423916535926, 'age_transmission_rate_15': 0.014789671573762105, 'age_transmission_rate_20': 0.9920476783771025, 'age_transmission_rate_25': 0.1248770171532827, 'age_transmission_rate_30': 0.28248803171794373, 'age_transmission_rate_35': 0.10759251319719022, 'age_transmission_rate_40': 0.12334553436526423, 'age_transmission_rate_45': 0.1167430899538966, 'age_transmission_rate_50': 0.0569642324681308, 'age_transmission_rate_55': 0.10247238315710167, 'age_transmission_rate_60': 0.30890550492350916, 'incubation_period': 5.4, 'infectious_period': 7.3}


In [38]:
Infec = [f"IXage_{i}" for i in range(0,65,5)]


In [39]:
total_cases_pred_DEM = map_res_smc[Infec].sum(axis=1)
total_cases_pred_SMC = map_res_smc[Infec].sum(axis=1)

In [42]:
total_cases = total_cases["Aug 2020":"Nov 2020"].iloc[::14]

In [43]:
plot_start_date = datetime(2020, 8, 1)
analysis_end_date = datetime(2020, 11, 30)

#plot= map_res["IXage_5"].plot()
plot = total_cases_pred_SMC.plot()
plot.update_xaxes(range=(plot_start_date, analysis_end_date))
# plot.add_trace(go.Scatter(x=df.loc["05_09"].index, y=df.loc["05_09"]["cases"], mode='markers', name='observed'))
plot.add_trace(go.Scatter(x=total_cases.index, y=total_cases["cases"], mode='markers', name='total_cases'))
# plot.add_trace(go.Scatter(x=total_cases.index, y=total_cases_pred_DEM, name='DEM'))

